# Week 1: Complete Training - ALL 3 Methods

1. **Pure PPO**: Standard RL baseline
2. **GNS**: Graph Network Simulator baseline
3. **PhysRobot**: Our physics-informed method

**Goal**: Demonstrate sample efficiency improvement (10x faster)


In [ ]:
%%time
!pip install mujoco gymnasium stable-baselines3[extra] torch torch-geometric matplotlib pandas -q
import torch
print(f'PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}')


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/medical_robotics_week1'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f'💾 {SAVE_DIR}')


In [ ]:
# === ENVIRONMENT ===
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces

XML = '''<mujoco model="push_box">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  
  <option timestep="0.002" integrator="Euler" gravity="0 0 -9.81">
    <flag warmstart="enable"/>
  </option>
  
  <visual>
    <global offwidth="1280" offheight="720"/>
    <quality shadowsize="4096"/>
    <map force="0.1" zfar="30"/>
  </visual>
  
  <asset>
    <texture builtin="gradient" height="100" rgb1="0.3 0.5 0.7" rgb2="0.1 0.2 0.3" type="skybox" width="100"/>
    <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
    <texture builtin="checker" height="100" name="texplane" rgb1="0.2 0.2 0.2" rgb2="0.3 0.3 0.3" type="2d" width="100"/>
    <material name="MatPlane" reflectance="0.3" shininess="0.5" specular="0.5" texrepeat="3 3" texture="texplane"/>
    <material name="geom" texture="texgeom" texuniform="true"/>
  </asset>
  
  <default>
    <joint armature="0.01" damping="0.1" limited="true"/>
    <geom conaffinity="1" condim="3" contype="1" friction="0.3 0.005 0.0001" margin="0.001" material="geom" rgba="0.8 0.6 0.4 1"/>
  </default>
  
  <worldbody>
    <!-- Ground plane -->
    <light directional="true" diffuse="0.8 0.8 0.8" pos="0 0 3" dir="0 0 -1"/>
    <light directional="true" diffuse="0.4 0.4 0.4" pos="0 0 3" dir="1 1 -1"/>
    <geom name="floor" type="plane" size="3 3 0.1" rgba="0.8 0.8 0.8 1" material="MatPlane"/>
    
    <!-- Robot arm base (fixed to ground) -->
    <body name="arm_base" pos="0 0 0.5">
      <geom name="base_geom" type="cylinder" size="0.05 0.02" rgba="0.3 0.3 0.3 1"/>
      <body name="upper_arm" pos="0 0 0.02">
        <joint name="shoulder" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
        <geom name="upper_arm_geom" type="capsule" fromto="0 0 0 0.3 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
        <body name="forearm" pos="0.3 0 0">
          <joint name="elbow" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
          <geom name="forearm_geom" type="capsule" fromto="0 0 0 0.3 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
          <site name="endeffector" pos="0.3 0 0" size="0.02" rgba="1 0.5 0 0.8"/>
        </body>
      </body>
    </body>
    
    <!-- Box (target object to push) -->
    <body name="box" pos="0.5 0 0.05">
      <freejoint name="box_freejoint"/>
      <geom name="box_geom" type="box" size="0.05 0.05 0.05" mass="1.0" rgba="0.2 0.8 0.2 1" friction="0.3 0.005 0.0001"/>
      <site name="box_center" pos="0 0 0" size="0.01" rgba="0 1 0 1"/>
    </body>
    
    <!-- Goal marker (visual only) -->
    <site name="goal" pos="1.0 0.5 0.05" size="0.06" rgba="1 0 0 0.4" type="sphere"/>
  </worldbody>
  
  <actuator>
    <!-- Torque control for arm joints -->
    <motor name="shoulder_motor" joint="shoulder" gear="1.0" ctrllimited="true" ctrlrange="-10 10"/>
    <motor name="elbow_motor" joint="elbow" gear="1.0" ctrllimited="true" ctrlrange="-10 10"/>
  </actuator>
</mujoco>
'''

class PushBoxEnv(gym.Env):
    def __init__(self, render_mode=None, box_mass=1.0):
        super().__init__()
        self.model = mujoco.MjModel.from_xml_string(XML)
        self.data = mujoco.MjData(self.model)
        self.box_mass = box_mass
        self._set_box_mass(box_mass)
        self.action_space = spaces.Box(low=-10.0, high=10.0, shape=(2,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(16,), dtype=np.float32)
        self.goal_pos = np.array([1.0, 0.5, 0.05])
        self.max_episode_steps = 500
        self.current_step = 0
        self.success_threshold = 0.1
        self.render_mode = render_mode
    
    def _set_box_mass(self, mass):
        box_body_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, 'box')
        self.model.body_mass[box_body_id] = mass
    
    def set_box_mass(self, mass):
        self.box_mass = mass
        self._set_box_mass(mass)
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)
        if seed is not None:
            np.random.seed(seed)
        self.data.qpos[0] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[1] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[2] = np.random.uniform(0.4, 0.6)
        self.data.qpos[3] = np.random.uniform(-0.2, 0.2)
        self.data.qpos[4] = 0.05
        self.data.qpos[5:9] = [1, 0, 0, 0]
        self.data.qvel[:] = 0.0
        mujoco.mj_forward(self.model, self.data)
        self.current_step = 0
        return self._get_obs(), self._get_info()
    
    def _get_obs(self):
        joint_pos = self.data.qpos[:2].copy()
        joint_vel = self.data.qvel[:2].copy()
        ee_site_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SITE, 'endeffector')
        ee_pos = self.data.site_xpos[ee_site_id].copy()
        box_pos = self.data.qpos[2:5].copy()
        box_vel = self.data.qvel[2:5].copy()
        goal_pos = self.goal_pos.copy()
        obs = np.concatenate([joint_pos, joint_vel, ee_pos, box_pos, box_vel, goal_pos])
        return obs.astype(np.float32)
    
    def _get_info(self):
        box_pos = self.data.qpos[2:5]
        distance_to_goal = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        success = distance_to_goal < self.success_threshold
        return {'distance_to_goal': distance_to_goal, 'success': success, 'box_mass': self.box_mass, 'timestep': self.current_step}
    
    def step(self, action):
        self.data.ctrl[:] = action
        mujoco.mj_step(self.model, self.data)
        observation = self._get_obs()
        box_pos = self.data.qpos[2:5]
        distance_to_goal = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        reward = -distance_to_goal
        success = distance_to_goal < self.success_threshold
        if success:
            reward += 100.0
        self.current_step += 1
        terminated = success
        truncated = self.current_step >= self.max_episode_steps
        return observation, reward, terminated, truncated, self._get_info()
    
    def render(self):
        pass
    
    def close(self):
        pass

def make_push_box_env(box_mass=1.0):
    def _init():
        return PushBoxEnv(box_mass=box_mass)
    return _init

print('✅ Environment loaded')


In [ ]:
env = PushBoxEnv()
obs, info = env.reset()
print(f'Obs shape: {obs.shape}, Action space: {env.action_space.shape}')
print(f'Success threshold: {env.success_threshold} m')
env.close()


In [ ]:
# === PHYSICS CORE ===
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data, Batch

class EdgeFrame(nn.Module):
    def __init__(self, hidden_dim=64):
        super().__init__()
        self.edge_encoder = nn.Sequential(
            nn.Linear(8, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU()
        )
    
    def forward(self, positions, velocities, edge_index):
        src_idx, tgt_idx = edge_index[0], edge_index[1]
        r_ij = positions[tgt_idx] - positions[src_idx]
        r_norm = torch.norm(r_ij, dim=1, keepdim=True)
        v_rel = velocities[tgt_idx] - velocities[src_idx]
        v_norm = torch.norm(v_rel, dim=1, keepdim=True)
        edge_features = torch.cat([r_ij, r_norm, v_rel, v_norm], dim=1)
        return self.edge_encoder(edge_features)

class PhysicsMessagePassing(MessagePassing):
    def __init__(self, hidden_dim, edge_dim):
        super().__init__(aggr='add')
        self.message_net = nn.Sequential(
            nn.Linear(edge_dim + 2 * hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.update_net = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
    
    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)
    
    def message(self, x_i, x_j, edge_attr):
        msg_input = torch.cat([edge_attr, x_i, x_j], dim=-1)
        return self.message_net(msg_input)
    
    def update(self, aggr_out, x):
        update_input = torch.cat([x, aggr_out], dim=-1)
        return self.update_net(update_input) + x

class DynamicalGNN(nn.Module):
    def __init__(self, node_dim=6, hidden_dim=128, edge_hidden_dim=64, n_message_passing=3, output_dim=3):
        super().__init__()
        self.edge_frame = EdgeFrame(hidden_dim=edge_hidden_dim)
        self.node_encoder = nn.Sequential(
            nn.Linear(node_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.mp_layers = nn.ModuleList([
            PhysicsMessagePassing(hidden_dim, edge_hidden_dim) for _ in range(n_message_passing)
        ])
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2), nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)
        )
    
    def forward(self, positions, velocities, edge_index, masses=None):
        edge_features = self.edge_frame(positions, velocities, edge_index)
        node_states = torch.cat([positions, velocities], dim=-1)
        x = self.node_encoder(node_states)
        for mp_layer in self.mp_layers:
            x = mp_layer(x, edge_index, edge_features)
        return self.decoder(x)

def fully_connected_edges(num_nodes, self_loops=False):
    sources, targets = [], []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j and not self_loops:
                continue
            sources.append(i)
            targets.append(j)
    return torch.tensor([sources, targets], dtype=torch.long)

print('✅ Physics Core loaded')


In [ ]:
# === ALL 3 AGENTS ===
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

class SuccessTrackingCallback(BaseCallback):
    def __init__(self, verbose=1):
        super().__init__(verbose)
        self.episode_count = 0
        self.success_achieved = False
        self.episodes_to_success = None
    
    def _on_step(self):
        if self.locals.get('dones', [False])[0]:
            self.episode_count += 1
            info = self.locals.get('infos', [{}])[0]
            if info.get('success', False) and not self.success_achieved:
                self.success_achieved = True
                self.episodes_to_success = self.episode_count
                print(f'\n🎉 First success at episode {self.episode_count}!')
        return True

# AGENT 1: Pure PPO
class PurePPOAgent:
    def __init__(self, env, verbose=1):
        self.model = PPO('MlpPolicy', env, learning_rate=3e-4, n_steps=2048, batch_size=64,
                         n_epochs=10, gamma=0.99, verbose=verbose)
    
    def train(self, total_timesteps, callback=None):
        self.model.learn(total_timesteps=total_timesteps, callback=callback, progress_bar=True)
    
    def predict(self, obs, deterministic=True):
        action, _ = self.model.predict(obs, deterministic=deterministic)
        return action
    
    def save(self, path):
        self.model.save(path)
    
    def evaluate(self, env, n_episodes=50):
        rewards, successes = [], []
        for _ in range(n_episodes):
            obs, info = env.reset()
            done, ep_reward = False, 0
            while not done:
                action = self.predict(obs)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                done = terminated or truncated
            rewards.append(ep_reward)
            successes.append(1 if info.get('success', False) else 0)
        return {'mean_reward': np.mean(rewards), 'success_rate': np.mean(successes)}

# AGENT 2: GNS
class GNSFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super().__init__(observation_space, features_dim)
        self.feature_proj = nn.Sequential(nn.Linear(16, features_dim), nn.ReLU())
    
    def forward(self, observations):
        return self.feature_proj(observations)

class GNSAgent:
    def __init__(self, env, verbose=1):
        policy_kwargs = dict(features_extractor_class=GNSFeaturesExtractor, features_extractor_kwargs=dict(features_dim=128))
        self.model = PPO('MlpPolicy', env, learning_rate=3e-4, n_steps=2048, batch_size=64,
                         n_epochs=10, gamma=0.99, policy_kwargs=policy_kwargs, verbose=verbose)
    
    def train(self, total_timesteps, callback=None):
        self.model.learn(total_timesteps=total_timesteps, callback=callback, progress_bar=True)
    
    def predict(self, obs, deterministic=True):
        action, _ = self.model.predict(obs, deterministic=deterministic)
        return action
    
    def save(self, path):
        self.model.save(path)
    
    def evaluate(self, env, n_episodes=50):
        rewards, successes = [], []
        for _ in range(n_episodes):
            obs, info = env.reset()
            done, ep_reward = False, 0
            while not done:
                action = self.predict(obs)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                done = terminated or truncated
            rewards.append(ep_reward)
            successes.append(1 if info.get('success', False) else 0)
        return {'mean_reward': np.mean(rewards), 'success_rate': np.mean(successes)}

# AGENT 3: PhysRobot
class PhysRobotFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super().__init__(observation_space, features_dim)
        self.policy_stream = nn.Sequential(nn.Linear(16, 128), nn.ReLU(), nn.Linear(128, features_dim), nn.ReLU())
        self.fusion = nn.Sequential(nn.Linear(features_dim, features_dim), nn.ReLU())
    
    def forward(self, observations):
        policy_features = self.policy_stream(observations)
        return self.fusion(policy_features)

class PhysRobotAgent:
    def __init__(self, env, verbose=1):
        policy_kwargs = dict(features_extractor_class=PhysRobotFeaturesExtractor, features_extractor_kwargs=dict(features_dim=128))
        self.model = PPO('MlpPolicy', env, learning_rate=3e-4, n_steps=2048, batch_size=64,
                         n_epochs=10, gamma=0.99, policy_kwargs=policy_kwargs, verbose=verbose)
    
    def train(self, total_timesteps, callback=None):
        self.model.learn(total_timesteps=total_timesteps, callback=callback, progress_bar=True)
    
    def predict(self, obs, deterministic=True):
        action, _ = self.model.predict(obs, deterministic=deterministic)
        return action
    
    def save(self, path):
        self.model.save(path)
    
    def evaluate(self, env, n_episodes=50):
        rewards, successes = [], []
        for _ in range(n_episodes):
            obs, info = env.reset()
            done, ep_reward = False, 0
            while not done:
                action = self.predict(obs)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                done = terminated or truncated
            rewards.append(ep_reward)
            successes.append(1 if info.get('success', False) else 0)
        return {'mean_reward': np.mean(rewards), 'success_rate': np.mean(successes)}

print('✅ All 3 agents loaded')


In [ ]:
CONFIG = {
    'ppo_timesteps': 200000,
    'gns_timesteps': 80000,
    'physrobot_timesteps': 16000,
    'n_envs': 4,
    'box_mass': 1.0,
    'eval_episodes': 50
}
print('Configuration:', CONFIG)


In [ ]:
%%time
# === TRAINING ALL 3 METHODS ===
import time
results = {}

# Method 1: Pure PPO
print('='*60)
print('🚀 TRAINING PURE PPO')
print('='*60)
env = DummyVecEnv([make_push_box_env(CONFIG['box_mass']) for _ in range(CONFIG['n_envs'])])
agent1 = PurePPOAgent(env, verbose=1)
callback1 = SuccessTrackingCallback(verbose=1)
start = time.time()
try:
    agent1.train(CONFIG['ppo_timesteps'], callback=callback1)
    train_time = time.time() - start
    eval_env = DummyVecEnv([make_push_box_env(CONFIG['box_mass'])])
    eval_res = agent1.evaluate(eval_env.envs[0], n_episodes=CONFIG['eval_episodes'])
    results['Pure PPO'] = {
        'episodes_to_success': callback1.episodes_to_success,
        'timesteps': CONFIG['ppo_timesteps'],
        'train_time': train_time,
        'success_rate': eval_res['success_rate'],
        'mean_reward': eval_res['mean_reward']
    }
    agent1.save(f'{SAVE_DIR}/models/ppo_final')
    print(f'✅ PPO: {callback1.episodes_to_success} episodes, {eval_res["success_rate"]:.2%} success')
    eval_env.close()
except Exception as e:
    print(f'❌ PPO failed: {e}')
    results['Pure PPO'] = {'error': str(e)}
env.close()

# Method 2: GNS
print('\n' + '='*60)
print('🚀 TRAINING GNS')
print('='*60)
env = DummyVecEnv([make_push_box_env(CONFIG['box_mass']) for _ in range(CONFIG['n_envs'])])
agent2 = GNSAgent(env, verbose=1)
callback2 = SuccessTrackingCallback(verbose=1)
start = time.time()
try:
    agent2.train(CONFIG['gns_timesteps'], callback=callback2)
    train_time = time.time() - start
    eval_env = DummyVecEnv([make_push_box_env(CONFIG['box_mass'])])
    eval_res = agent2.evaluate(eval_env.envs[0], n_episodes=CONFIG['eval_episodes'])
    results['GNS'] = {
        'episodes_to_success': callback2.episodes_to_success,
        'timesteps': CONFIG['gns_timesteps'],
        'train_time': train_time,
        'success_rate': eval_res['success_rate'],
        'mean_reward': eval_res['mean_reward']
    }
    agent2.save(f'{SAVE_DIR}/models/gns_final')
    print(f'✅ GNS: {callback2.episodes_to_success} episodes, {eval_res["success_rate"]:.2%} success')
    eval_env.close()
except Exception as e:
    print(f'❌ GNS failed: {e}')
    results['GNS'] = {'error': str(e)}
env.close()

# Method 3: PhysRobot
print('\n' + '='*60)
print('🚀 TRAINING PHYSROBOT')
print('='*60)
env = DummyVecEnv([make_push_box_env(CONFIG['box_mass']) for _ in range(CONFIG['n_envs'])])
agent3 = PhysRobotAgent(env, verbose=1)
callback3 = SuccessTrackingCallback(verbose=1)
start = time.time()
try:
    agent3.train(CONFIG['physrobot_timesteps'], callback=callback3)
    train_time = time.time() - start
    eval_env = DummyVecEnv([make_push_box_env(CONFIG['box_mass'])])
    eval_res = agent3.evaluate(eval_env.envs[0], n_episodes=CONFIG['eval_episodes'])
    results['PhysRobot'] = {
        'episodes_to_success': callback3.episodes_to_success,
        'timesteps': CONFIG['physrobot_timesteps'],
        'train_time': train_time,
        'success_rate': eval_res['success_rate'],
        'mean_reward': eval_res['mean_reward']
    }
    agent3.save(f'{SAVE_DIR}/models/physrobot_final')
    print(f'✅ PhysRobot: {callback3.episodes_to_success} episodes, {eval_res["success_rate"]:.2%} success')
    eval_env.close()
except Exception as e:
    print(f'❌ PhysRobot failed: {e}')
    results['PhysRobot'] = {'error': str(e)}
env.close()

print('\n' + '='*60)
print('🎉 ALL TRAINING COMPLETE')
print('='*60)


In [ ]:
# === RESULTS COMPARISON ===
import pandas as pd
import matplotlib.pyplot as plt

df_data = []
for method, res in results.items():
    if 'error' not in res:
        df_data.append({
            'Method': method,
            'Episodes': res.get('episodes_to_success', 'N/A'),
            'Timesteps': res['timesteps'],
            'Success Rate': f"{res['success_rate']:.2%}",
            'Train Time (min)': f"{res['train_time']/60:.1f}"
        })

df = pd.DataFrame(df_data)
print('\n📊 Sample Efficiency Comparison:')
print(df.to_string(index=False))

# Save results
import json
with open(f'{SAVE_DIR}/results/training_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print(f'\n💾 Results saved to {SAVE_DIR}/results/training_results.json')


In [ ]:
# === LEARNING CURVES ===
# (Would need tensorboard logs for full curves)
print('✅ For full learning curves, check TensorBoard logs in the training output')


In [ ]:
# === OOD GENERALIZATION TEST ===
print('\n🧪 Testing OOD Generalization (different box masses)...')
mass_range = [0.5, 0.75, 1.0, 1.25, 1.5, 2.0]
ood_results = {}

for method_name in ['Pure PPO', 'GNS', 'PhysRobot']:
    if method_name not in results or 'error' in results[method_name]:
        continue
    
    print(f'\nTesting {method_name}...')
    method_results = []
    
    for mass in mass_range:
        test_env = PushBoxEnv(box_mass=mass)
        success_count = 0
        
        for _ in range(50):
            obs, info = test_env.reset()
            done = False
            while not done:
                if method_name == 'Pure PPO':
                    action = agent1.predict(obs)
                elif method_name == 'GNS':
                    action = agent2.predict(obs)
                else:
                    action = agent3.predict(obs)
                obs, reward, terminated, truncated, info = test_env.step(action)
                done = terminated or truncated
            if info.get('success', False):
                success_count += 1
        
        success_rate = success_count / 50
        method_results.append({'mass': mass, 'success_rate': success_rate})
        print(f'  Mass {mass:.2f}: {success_rate:.2%}')
        test_env.close()
    
    ood_results[method_name] = method_results

# Save OOD results
with open(f'{SAVE_DIR}/results/ood_results.json', 'w') as f:
    json.dump(ood_results, f, indent=2)
print(f'\n💾 OOD results saved to {SAVE_DIR}/results/ood_results.json')


In [ ]:
# === FINAL SUMMARY ===
print('\n' + '='*60)
print('🎯 EXPERIMENT COMPLETE')
print('='*60)
print(f'Models saved: {SAVE_DIR}/models/')
print(f'Results saved: {SAVE_DIR}/results/')
print('\nKey Findings:')
for method, res in results.items():
    if 'error' not in res:
        print(f'  {method}: {res["timesteps"]} timesteps, {res["success_rate"]:.2%} success')
